In [1]:
import networkx as nx
import numpy as np
import os
import random as rand
from collections import defaultdict

In [2]:
def get_r_dictionaries(file,mapping=True):
    with open(file, "r") as in_file:
        dict_map={}
        for line in in_file:
            line.strip()
            if mapping:
                dict_map[line.split()[1]]=line.split()[0]
            else:
                dict_map[line.split()[0]]=set(line.split()[1:])
    return dict_map

In [3]:
def get_cyclic_net(filename):
    G=nx.read_edgelist(filename, comments='#', delimiter="\t", nodetype =str,  data=(('mode',str),), create_using=nx.DiGraph())
    G.remove_nodes_from(["Source", "Target"])
    selfloops=G.selfloop_edges()
    G.remove_edges_from(G.selfloop_edges())

    while 0 in [d[1] for d in G.in_degree()] or 0 in [d[1] for d in G.out_degree()]:
        nodes_to_remove=[node for node in G if G.in_degree(node) == 0 or G.out_degree(node) == 0]
        G.remove_nodes_from(nodes_to_remove)
        
        
    selfloops_in_reservoir=[edge for edge in selfloops if edge[0] in G.nodes()]
    G.add_edges_from(selfloops_in_reservoir)

    return G

In [4]:
def build_adj_weighted_matrix(filename,mapping):
        #NETWORK v2.0
    net=get_cyclic_net(filename)
    nx.relabel_nodes(net,mapping,copy=False)
    dict_pos=dict((node,pos) for (pos,node) in enumerate(net.nodes()))
    for edge in net.edges(data="mode", default=0):
        source,target,mode=edge
        if mode== "+":
            net[source][target]["weight"]= abs(rand.gauss(0,1))
        elif mode== "-":
            net[source][target]["weight"]= rand.gauss(0,1)*-1
        elif mode== 0:
            net[source][target]["weight"]= rand.gauss(0,1)
        
    return nx.to_numpy_matrix(net),dict_pos

In [5]:
def write_nodes_file(out_file,filename,net):
    with open("%s_%s"%(out_file,filename),"w") as out:
        for gene in net.nodes():
            if "hsa" in gene:
                gene=gene[4:].lower()
            out.write(gene +"\n")

In [6]:
file="Dataset1/network_edge_list_ENCODE.csv"
net=get_cyclic_net("Dataset1/network_edge_list_ENCODE.csv")
G=nx.read_edgelist(file, comments='#', delimiter="\t", nodetype =str,  data=(('mode',str),), create_using=nx.DiGraph())

In [7]:
readout_direct_targets=defaultdict(set)

for source, target in G.edges():
    if source in net.nodes() and target not in net.nodes():
        readout_direct_targets[source].add(target)

readout_direct_targets["MAX"]
    #print(target)

{'AATF',
 'ABCA17P',
 'ABCA3',
 'ABCB4',
 'ABCF1',
 'ABCG1',
 'AC011472.1',
 'AC013553.1',
 'AC044799.1',
 'AC136618.1',
 'ACAD9',
 'ACAP1',
 'ACTC1',
 'ADAM19',
 'ADAMTS16',
 'ADH6',
 'ADM',
 'ADORA1',
 'AFMID',
 'AFP',
 'AGFG2',
 'AGT',
 'AIMP2',
 'AIP',
 'AK2',
 'AKR1A1',
 'AKR1C1',
 'AKR1C3',
 'AKT1S1',
 'ALB',
 'ALDH1A1',
 'ALDOC',
 'ALG2',
 'ALG3',
 'ALOX15B',
 'AMDHD2',
 'AMN',
 'AMPD3',
 'ANAPC7',
 'ANG',
 'ANK1',
 'ANKRD12',
 'ANKRD30BL',
 'ANO3',
 'ANP32B',
 'AOC3',
 'AP1G2',
 'AP1S1',
 'AP2M1',
 'AP4M1',
 'APBB3',
 'APEX1',
 'APOBR',
 'APOC1',
 'APOC2',
 'APOE',
 'APOH',
 'APRT',
 'ARHGAP24',
 'ARSK',
 'ASGR1',
 'ASGR2',
 'ASPSCR1',
 'ATL3',
 'ATP13A4',
 'ATP6V0B',
 'ATP6V0D1',
 'ATP8B1',
 'ATXN3',
 'BAT3',
 'BAX',
 'BCAS3',
 'BCKDHA',
 'BCL2',
 'BCR',
 'BIN1',
 'BLOC1S3',
 'BMP4',
 'BOLA1',
 'BPIL1',
 'BST2',
 'BUB3',
 'C10orf116',
 'C10orf18',
 'C10orf46',
 'C10orf76',
 'C10orf88',
 'C11orf59',
 'C12orf24',
 'C12orf49',
 'C13orf23',
 'C16orf5',
 'C16orf54',
 'C16orf73',
 '

In [8]:
len(set(net["SRF"]))

12

In [9]:
len(net.out_edges("SRF"))

12

In [10]:
len(set.union(*readout_direct_targets.values()))

13397

In [11]:
filename=file[file.index("list")+5:file.index(".csv")]

In [12]:
write_nodes_file("all_gene",filename,G)

In [13]:
len(net.nodes())

207

In [14]:
## Get R dictionaries
## GO term -> set ids
GO_id_map=get_r_dictionaries("test.txt",mapping=False)
#print(GO_id_map)

## id to ENTREZ id
edgeid_ezid_map=get_r_dictionaries("mapping_id_to_entrez.txt")
print(edgeid_ezid_map)

{'5669': 'SP1', '6667': 'SP1', '199699': 'SP1', '6668': 'SP2', '2623': 'GATA1', '672': 'BRCA1', '3172': 'HNF4A', '1876': 'E2F6', '406920': 'hsa-miR-130b', '4208': 'MEF2C', '140690': 'CTCFL', '5452': 'POU2F2', '406982': 'hsa-miR-20a', '406953': 'hsa-miR-18a', '7702': 'ZNF143', '23462': 'HEY1', '6721': 'SREBF2', '25942': 'SIN3A', '2355': 'FOSL2', '53335': 'BCL11A', '7182': 'NR2C2', '147912': 'SIX5', '4149': 'MAX', '4601': 'MXI1', '10891': 'PPARGC1A', '4801': 'NFYB', '1958': 'EGR1', '1051': 'CEBPB', '5993': 'RFX5', '6688': 'SPI1', '9774': 'BCLAF1', '6938': 'TCF12', '639': 'PRDM1', '3066': 'HDAC2', '6256': 'RXRA', '4779': 'NRF1', '4899': 'NRF1', '6886': 'TAL1', '6925': 'TCF4', '6934': 'TCF4', '2099': 'ESR1', '905': 'CCNT2', '10664': 'CTCF', '5090': 'PBX3', '11128': 'POLR3A', '10009': 'ZBTB33', '3659': 'IRF1', '2033': 'EP300', '2113': 'ETS1', '2908': 'NR3C1', '51341': 'ZBTB7A', '1869': 'E2F1', '6722': 'SRF', '3661': 'IRF3', '2353': 'FOS', '8061': 'FOSL1', '3726': 'JUNB', '10535': 'JUNB', '6

In [15]:
## Primero: Hay que cambiarle el id del res al id de ENTREZ
## hay que hacer un mapping dictionary con la info que tenemos de los nodos de GO.term
##Como no todos están anotados es mejor crear un diccionario 
mapping_relabel = edgeid_ezid_map
for node in net.nodes():
    if node not in edgeid_ezid_map.values():
        mapping_relabel[node]=node
mapping_relabel

{'10009': 'ZBTB33',
 '100126319': 'hsa-miR-216b',
 '100126340': 'hsa-miR-944',
 '100126348': 'hsa-miR-760',
 '100302143': 'hsa-miR-1248',
 '100302201': 'hsa-miR-1228',
 '100302232': 'hsa-miR-1226',
 '10127': 'ZNF263',
 '10155': 'TRIM28',
 '1051': 'CEBPB',
 '10535': 'JUNB',
 '10538': 'BATF',
 '10664': 'CTCF',
 '10891': 'PPARGC1A',
 '1106': 'CHD2',
 '11128': 'POLR3A',
 '140690': 'CTCFL',
 '147912': 'SIX5',
 '1488': 'CTBP2',
 '1826': 'CHD2',
 '1869': 'E2F1',
 '1874': 'E2F4',
 '1876': 'E2F6',
 '1879': 'EBF1',
 '1958': 'EGR1',
 '199699': 'SP1',
 '1997': 'ELF1',
 '2005': 'ELK4',
 '2033': 'EP300',
 '2099': 'ESR1',
 '2113': 'ETS1',
 '23462': 'HEY1',
 '23512': 'SUZ12',
 '2353': 'FOS',
 '2355': 'FOSL2',
 '23764': 'MAFF',
 '2551': 'GABPA',
 '25942': 'SIN3A',
 '2623': 'GATA1',
 '2624': 'GATA2',
 '26469': 'BDP1',
 '2908': 'NR3C1',
 '2959': 'GTF2B',
 '3066': 'HDAC2',
 '3169': 'FOXA1',
 '3170': 'FOXA2',
 '3172': 'HNF4A',
 '3174': 'HNF4G',
 '3297': 'HSF1',
 '3659': 'IRF1',
 '3661': 'IRF3',
 '3662': 'I

In [16]:
for key,values in GO_id_map.items():
    GO_id_map[key]=set([mapping_relabel[value] for value in values])
GO_id_map

{'GO:0051591': {'FOS', 'FOSL1', 'JUN', 'JUNB', 'JUND', 'SREBF1', 'STAT1'},
 'GO:0070317': {'BRCA1', 'E2F1', 'E2F6', 'MAX', 'SUZ12'}}

In [17]:
set.union(*GO_id_map.values())

{'BRCA1',
 'E2F1',
 'E2F6',
 'FOS',
 'FOSL1',
 'JUN',
 'JUNB',
 'JUND',
 'MAX',
 'SREBF1',
 'STAT1',
 'SUZ12'}

In [18]:
readout_common=set(gene for gene_res in set.union(*GO_id_map.values()) for gene in set(G[gene_res]) if gene in set.union(*readout_direct_targets.values()) )

In [19]:
readout_common

{'SNORD114-24',
 'HLCS',
 'BLM',
 'TRIP13',
 'KIAA0101',
 'C2orf44',
 'ZIC4',
 'DHX58',
 'APBB3',
 'TRIM34',
 'LHX9',
 'CD164',
 'NR4A1',
 'STX6',
 'PES1',
 'TARS2',
 'SNORD104',
 'hsa-miR-17*',
 'KCNJ4',
 'CTDSPL2',
 'RND1',
 'FAM24B',
 'LRRC61',
 'TMEM14C',
 'RPA2',
 'IRX5',
 'RIF1',
 'DMRTA2',
 'UBE2C',
 'ENOPH1',
 'RNF32',
 'ONECUT1',
 'GBP4',
 'DHRS13',
 'STARD4',
 'ATP13A4',
 'MDP1',
 'TMEM126B',
 'NR2F1',
 'RUVBL1',
 'LBH',
 'RPS12',
 'AFMID',
 'ARPP19',
 'LMO7',
 'LIMS2',
 'CLNK',
 'hsa-miR-196b',
 'IMMP1L',
 'HIF1A',
 'SERBP1',
 'LMBRD2',
 'PIKFYVE',
 'CCDC99',
 'KIAA0947',
 'HAVCR2',
 'GPCPD1',
 'PFDN4',
 'RFWD3',
 'C1orf156',
 'SFRS15',
 'MSH2',
 'PIGX',
 'OSR2',
 'ARL16',
 'ACTC1',
 'RPL28',
 'ANG',
 'C19orf62',
 'WDR65',
 'ZNF341',
 'CCND3',
 'TIPIN',
 'EML4',
 'MRPS18C',
 'HRC',
 'AGRN',
 'LOC645166',
 'SBSN',
 'BPGM',
 'MRP63',
 'C2orf34',
 'C21orf49',
 'GTF2A1',
 'SELL',
 'TPP1',
 'LGALS7',
 'PPP1CB',
 'TESC',
 'SEPT5',
 'TLR10',
 'OTP',
 'HOXA11AS',
 'SC5DL',
 'HSPA5',

In [20]:
def get_len(item):
    return len(net.in_edges(item))

In [21]:
lista=[(gene,len(G.in_edges(gene))) for gene in readout_common]

In [22]:
len_innodes_gene=defaultdict(set)
for el in lista:
    len_innodes_gene[el[1]].add(el[0])
len_innodes_gene[5]

{'AARS',
 'AATF',
 'ABCA17P',
 'ABCA3',
 'ABCB8',
 'ACAD9',
 'ACTL6A',
 'AIFM2',
 'AIMP2',
 'ALDH1A1',
 'APTX',
 'ARRDC2',
 'ATP5G1',
 'AVPI1',
 'BRD9',
 'C10orf88',
 'C11orf59',
 'C13orf29',
 'C14orf119',
 'C16orf74',
 'C19orf40',
 'C19orf51',
 'C1QTNF6',
 'C1orf107',
 'C1orf228',
 'C20orf111',
 'C20orf24',
 'C21orf58',
 'C5orf39',
 'C5orf51',
 'C6orf89',
 'C7orf40',
 'C8orf55',
 'C9orf3',
 'CABC1',
 'CCBL1',
 'CCDC123',
 'CCNT1',
 'CDC5L',
 'CDR2',
 'CENPL',
 'CISD2',
 'CNTLN',
 'COX5A',
 'CPB2',
 'CREB3L4',
 'CSPG4',
 'CYP51A1',
 'DAGLB',
 'DARS2',
 'DDIT3',
 'DDX47',
 'DENND4A',
 'DEPDC4',
 'DLX4',
 'DNA2',
 'DNAJC9',
 'DOCK10',
 'DRAP1',
 'DSCC1',
 'ENOPH1',
 'ERGIC2',
 'EXD1',
 'FLJ31306',
 'FLJ45983',
 'FOXE1',
 'GADD45A',
 'GIMAP4',
 'GLDC',
 'GPATCH3',
 'GPBP1',
 'HELQ',
 'HES4',
 'HGSNAT',
 'HIGD2A',
 'HMX2',
 'HOXA2',
 'HOXD8',
 'HRC',
 'HSF2BP',
 'HSPBAP1',
 'ICAM1',
 'IFI6',
 'IQCD',
 'ISLR2',
 'ITPR3',
 'KDM3A',
 'KEAP1',
 'KIAA0406',
 'KNCN',
 'LBX1',
 'LBX2',
 'LENG1',


In [23]:
set_common=[]
for GO_term in GO_id_map.keys():
    print()
    print(GO_term)
    for gene_view in GO_id_map[GO_term]:
        print(gene_view,"",set(G[gene_view]) & len_innodes_gene[5])
        set_common.append(set(G[gene_view]) & len_innodes_gene[5])


GO:0070317
SUZ12  {'WT1', 'NKX2-5', 'hsa-miR-9*', 'SIM1', 'SIX1', 'LHX5', 'FLJ45983', 'ISLR2', 'LBX2', 'OSR2', 'HOXD8', 'SP9', 'AATF', 'FOXE1', 'HMX2', 'LBX1', 'DLX4', 'HOXA2'}
BRCA1  {'NOL11', 'SLC15A4', 'SNORD35B', 'STYXL1', 'ERGIC2', 'POLR2E', 'GPBP1', 'GPATCH3', 'CDC5L', 'SLC25A11', 'XPC', 'C6orf89', 'THADA', 'LSMD1', 'MED22', 'NUDT2', 'CCNT1'}
MAX  {'HSF2BP', 'C11orf59', 'NOC3L', 'PNO1', 'POLR2E', 'SNORA76', 'LOXL1', 'AIMP2', 'AATF', 'THADA', 'SGOL1', 'HIGD2A', 'CABC1', 'TASP1', 'ACAD9', 'HRC', 'SCAMP3', 'RPS8', 'LONP1', 'EXD1', 'POLE4', 'UBE2S', 'ENOPH1', 'PLD6', 'GLDC', 'ABCA3', 'HSPBAP1', 'DOCK10', 'WDR36', 'HELQ', 'THAP8', 'ABCA17P', 'ZFYVE27', 'MAP1D', 'SAE1', 'MCOLN1', 'C7orf40', 'KDM3A', 'WDR89', 'C1orf107', 'ALDH1A1', 'C10orf88', 'ZNF785', 'HES4', 'WDR4', 'CCNT1'}
E2F6  {'KIAA0406', 'HSF2BP', 'PQLC2', 'AIMP2', 'AATF', 'CABC1', 'ACAD9', 'HRC', 'SP9', 'KEAP1', 'EXD1', 'POLE4', 'ENOPH1', 'CNTLN', 'ABCA3', 'HSPBAP1', 'WDR36', 'ABCA17P', 'DNA2', 'RBM18', 'C8orf55', 'SDF4', 'C1

In [25]:
i=range(0,len(set_common))
for index,el in enumerate(set_common):
    print(index)
    for number in i:
        if number != index:
            print(el & set_common[number])

0
set()
{'C8orf55', 'HSF2BP', 'HES4', 'PQLC2'}
{'OSR2'}
{'HSF2BP', 'HES4'}
set()
{'CCBL1', 'HES4'}
set()
set()
set()
set()
set()
1
set()
set()
set()
{'THADA', 'CCNT1', 'POLR2E'}
set()
{'THADA', 'CDC5L'}
set()
set()
set()
set()
set()
2
{'C8orf55', 'HSF2BP', 'HES4', 'PQLC2'}
set()
{'AATF', 'SP9'}
{'C10orf88', 'AATF', 'HSF2BP', 'ABCA3', 'WDR36', 'HRC', 'HES4', 'AIMP2', 'CABC1', 'ENOPH1', 'POLE4', 'EXD1', 'ABCA17P', 'ACAD9', 'ZNF785', 'HSPBAP1'}
set()
{'ACAD9', 'HES4'}
set()
{'CABC1'}
set()
set()
set()
3
{'OSR2'}
set()
{'AATF', 'SP9'}
{'AATF'}
set()
set()
set()
set()
set()
set()
set()
4
{'HSF2BP', 'HES4'}
{'THADA', 'CCNT1', 'POLR2E'}
{'C10orf88', 'AATF', 'HSF2BP', 'ABCA3', 'WDR36', 'HRC', 'HES4', 'AIMP2', 'CABC1', 'ENOPH1', 'POLE4', 'EXD1', 'ABCA17P', 'ACAD9', 'ZNF785', 'HSPBAP1'}
{'AATF'}
set()
{'THADA', 'ACAD9', 'ZFYVE27', 'HES4'}
set()
{'CABC1'}
{'ALDH1A1'}
set()
set()
5
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
6
{'CCBL1', 'HES4'}
{'THADA', 'CDC5L'}
{'ACAD9', 'H

In [24]:
for gene in len_innodes_gene[5]:
    
    genes_res_no_go=set(edge[0] for edge in G.in_edges(gene) if edge[0] not in set.union(*GO_id_map.values()))
    
    if genes_res_no_go.intersection(set(net.nodes())) != genes_res_no_go:
        continue
    print(gene)
    for key in GO_id_map.keys():
        print(key)
        print(set(gene_res for edge in G.in_edges(gene) for gene_res in GO_id_map[key] if gene_res in edge[0] ))
   
    print(genes_res_no_go)
   
    print()
    
   

C11orf59
GO:0070317
{'MAX'}
GO:0051591
set()
{'USF2', 'ATF3', 'NFE2', 'USF1'}

RFPL4B
GO:0070317
set()
GO:0051591
{'JUND', 'JUN'}
{'STAT3', 'EP300', 'RFX5', 'CEBPB'}

STYXL1
GO:0070317
{'BRCA1'}
GO:0051591
set()
{'ELF1', 'ZBTB33', 'GABPA', 'ETS1'}

TRIP13
GO:0070317
{'E2F1'}
GO:0051591
set()
{'MYC', 'SIN3A', 'E2F4', 'GABPA'}

AVPI1
GO:0070317
{'E2F1'}
GO:0051591
set()
{'TFAP2C', 'TFAP2A', 'EGR1', 'NR3C1'}

DAGLB
GO:0070317
set()
GO:0051591
{'JUN', 'FOS', 'JUND'}
{'GTF2B', 'TAL1', 'NRF1', 'FOSL2'}

RBCK1
GO:0070317
set()
GO:0051591
{'SREBF1', 'JUND', 'JUN'}
{'EP300', 'SREBF2'}

SGOL1
GO:0070317
{'MAX'}
GO:0051591
set()
{'USF2', 'E2F4', 'NFE2', 'USF1'}

TMEM161A
GO:0070317
set()
GO:0051591
{'FOS'}
{'TCF4', 'PBX3', 'NFYB', 'NFYA'}

S100A6
GO:0070317
set()
GO:0051591
{'FOS'}
{'TFAP2C', 'TFAP2A', 'SMARCA4', 'SMARCB1'}

SNORD35B
GO:0070317
{'BRCA1'}
GO:0051591
set()
{'BCL3', 'SMARCA4', 'ZBTB33', 'ETS1'}

ACAD9
GO:0070317
{'MAX', 'E2F6'}
GO:0051591
{'JUND', 'JUN'}
{'ATF3', 'NRF1'}

TMEM14C
GO

In [37]:
GO_007={readout_gene : { edge[0] for edge in G.in_edges(readout_gene) if edge[0] in net.nodes()}for readout_gene in ["STYXL1","BRD9","ABCA3"]}
np.save('GO_007.npy',GO_007) 

In [41]:
GO_007_005= {readout_gene :{ edge[0] for edge in G.in_edges(readout_gene) if edge[0] in net.nodes()}for readout_gene in ["ALDH1A1","ACAD9"]}
np.save('GO_007and005.npy',GO_007_005) 

In [32]:
GO_005={readout_gene : { edge[0] for edge in G.in_edges(readout_gene)} for readout_gene in ["ZNF775","TMEM14C","C20orf111"]}
np.save('GO_005.npy',GO_005) 

In [ ]:
## Win test
res_size=207
in_size=1
i_scaling=1

In [ ]:
matrix,dict_pos=build_adj_weighted_matrix(file,mapping_relabel)

In [ ]:
Win=np.zeros((res_size,1+in_size))*i_scaling
Win[1,]

In [ ]:
#print(GO_id_map["GO:0030220"])
for gene in GO_id_map["GO:0030220"]:
    print(dict_pos[gene])
    Win[dict_pos[gene],]=2
print(np.where(Win==2))

In [ ]:
def input_matrix_just_genes_GOterm(Win,GOterm,GO_id_map):
    for gene in GO_id_map[GOterm]:
        Win[dict_pos[gene],]=np.random.uniform(0,1)
    return Win

In [ ]:
Win=np.zeros((res_size,1+in_size))*i_scaling
input_matrix_just_genes_GOterm(Win,"GO:0030220",GO_id_map)